In [3]:
import re
import time
import requests
from multiprocessing import Pool
from multiprocessing import freeze_support,Process

In [ ]:
def test(url):
    print(url[-2:])
    
#urls = ['https://www.qiushibaike.com/8hr/page/{}/'.format(str(i)) for i in range(1,36)]
urls_2 = ['asfdsadf','asdfdfrrt','fbnhfgf','fgsdfgfdgfdgdf']

if __name__ == '__main__':
    freeze_support()
    start_2 = time.time()
    print('start', start_2)
    pool = Pool(processes=2)
    print('a')
    pool.map(test, urls_2)
    print('b')
    end_2 = time.time()
    print('end:',end_2)
    print('2进程爬虫耗时：',end_2-start_2)


start 1539648373.5423157
a


In [ ]:
start_1 = time.time()
for url in urls_2:
    test(url)
end_1 = time.time()
print('串行爬虫耗时：',end_1-start_1)

In [1]:
from multiprocessing import Process
import os

# 子进程要执行的代码
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Process(target=run_proc, args=('test',))
    print('Child process will start.')
    p.start()
    p.join()
    print('Child process end.')


Parent process 16016.
Child process will start.
Child process end.


In [ ]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

Parent process 16016.
Waiting for all subprocesses done...


In [2]:
#这个可以运行
from multiprocessing import Process

def f(name):
    print('hello', name)

if __name__ == '__main__':
    p = Process(target=f, args=('bob',)) # 新建一个子进程p，目标函数是f，args是函数f的参数列表
    p.start() # 开始执行进程
    p.join() # 等待子进程结束

### 在多进程中使用全局变量会出现问题

In [5]:
import threading
import time
import random


def greet(index):
    print('hello world - %d') % index


def line_run():
    for x in range(5):
        greet(x)


def async_run():
    for x in range(5):
        th = treading.Thread(target=greet, args=[x])
        th.start()


MONEY = 0


def producer():
    while True:
        global MONEY
        random_money = random.randint(10, 100)
        MONEY += random_money
        print('生产者%s 生产了%d, 余额%d' % (threading.current_thread, random_money,
                                     MONEY))
        time.sleep(0.5)


def comstomer():
    while True:
        global MONEY
        random_money = random.randint(10, 100)
        if MONEY > random_money:
            MONEY -= random_money
            print('消费者%s-消费了：%d, 余额%d' % (threading.current_thread,
                                          random_money, MONEY))

        else:
            print('需要消费的钱为： %d, 余额为：%d' % (random_money, MONEY))
        time.sleep(0.5)


def p_c_test():
    for x in range(3):
        th = threading.Thread(target=producer)
        th.start()
    for x in range(3):
        th = threading.Thread(target=comstomer)
        th.start()


if __name__ == '__main__':
    p_c_test()


生产者<function current_thread at 0x0000022885A34BF8> 生产了92, 余额92
0000000000000000000000生产者<function current_thread at 0x0000022885A34BF8> 生产了95, 余额187
生产者<function current_thread at 0x0000022885A34BF8> 生产了42, 余额229消费者<function current_thread at 0x0000022885A34BF8>-消费了：89, 余额62

生产者<function current_thread at 0x0000022885A34BF8> 生产了29, 余额91

生产者<function current_thread at 0x0000022885A34BF8> 生产了62, 余额153消费者<function current_thread at 0x0000022885A34BF8>-消费了：68, 余额85

生产者<function current_thread at 0x0000022885A34BF8> 生产了69, 余额154
消费者<function current_thread at 0x0000022885A34BF8>-消费了：49, 余额105
消费者<function current_thread at 0x0000022885A34BF8>-消费了：47, 余额58
需要消费的钱为： 69, 余额为：58消费者<function current_thread at 0x0000022885A34BF8>-消费了：42, 余额16

7680 172 92

95 267
生产者<function current_thread at 0x0000022885A34BF8> 生产了25, 余额292
生产者<function current_thread at 0x0000022885A34BF8> 生产了88, 余额380
生产者<function current_thread at 0x0000022885A34BF8> 生产了88, 余额468
消费者<function current_thread at 0x000002288

### 所以需要添加进程锁

In [ ]:
import threading
import time
import random

def greet(index):
    print('hello world - %d') % index


def line_run():
    for x in range(5):
        greet(x)


def async_run():
    for x in range(5):
        th = treading.Thread(target=greet, args=[x])
        th.start()


MONEY = 0


def producer():
    while True:
        global MONEY
        random_money = random.randint(10, 100)
        MONEY += random_money
        print('生产者%s 生产了%d, 余额%d' % (threading.current_thread, random_money,
                                     MONEY))
        time.sleep(0.5)


def comstomer():
    while True:
        global MONEY
        random_money = random.randint(10, 100)
        if MONEY > random_money:
            MONEY -= random_money
            print('消费者%s-消费了：%d, 余额%d' % (threading.current_thread,
                                          random_money, MONEY))

        else:
            print('需要消费的钱为： %d, 余额为：%d' % (random_money, MONEY))
        time.sleep(0.5)


def p_c_test():
    for x in range(3):
        th = threading.Thread(target=producer)
        th.start()
    for x in range(3):
        th = threading.Thread(target=comstomer)
        th.start()


if __name__ == '__main__':
    p_c_test()